# Probando por separado

In [ ]:
from cmp.evaluation import evaluate_reverse_parse

from engine.Lexer.hulk_lexer import HulkLexer 
from engine.language.tokens_type import hulk_tokens
from engine.language.grammar import G
from engine.Parser.hulk_parser import HulkParser

hulk_lexer = HulkLexer(hulk_tokens, G.EOF)
hulk_parser = HulkParser(G)

In [ ]:
from tests.test_cases import test_cases

tokens = []
right_parses = []
operations = []
asts = []

for input in test_cases:
    token = hulk_lexer(input)
    tokens.append(token)

    right_parse, operation = hulk_parser(token)
    right_parses.append(right_parse)
    operations.append(operation)

    ast = evaluate_reverse_parse(right_parse, operation, token)
    asts.append(ast)

    print(token)
    print(right_parse)
    print(operation)
    print(ast)
    print("="*100)


In [ ]:

from engine.semantic.type_builder_visitor import TypeBuilder
# from engine.semantic.type_checker_visitor import TypeChecker
from engine.semantic.type_collector_visitor import TypeCollector
from engine.semantic.type_inferer_visitor import TypeInferer
from engine.semantic.variable_collector_visitor import VarCollector


def semantic_analysis_pipeline(ast, debug=False):
    # if debug:
    #     formatter = Formatter()
    #     formatted_ast = formatter.visit(ast)
    #     print('===================== AST =====================')
    #     print(formatted_ast)
    if debug:
        print('============== COLLECTING TYPES ===============')
    errors = []
    collector = TypeCollector(errors)
    collector.visit(ast)
    context = collector.context
    if debug:
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('=============== BUILDING TYPES ================')
    builder = TypeBuilder(context, errors)
    builder.visit(ast)
    if debug:
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('=============== CHECKING TYPES ================')
        print('---------------- COLLECTING VARIABLES ------------------')
    var_collector = VarCollector(context, errors)
    scope = var_collector.visit(ast)
    if debug:
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('Scope:')
        print(scope)
        print('---------------- INFERRING TYPES ------------------')
    type_inferrer = TypeInferer(context, errors)
    type_inferrer.visit(ast)

    # Check if there are any inference errors and change the types to ErrorType if there are
    # inference_errors = context.inference_errors() + scope.inference_errors()
    # errors.extend(inference_errors)
    if debug:
        # print('Iterations: ' + str(type_inferrer.current_iteration))
        print('Errors: [')
        for error in errors:
            print('\t', error)
        print(']')
        print('Context:')
        print(context)
        print('Scope:')
        print(scope)
        print('---------------- CHECKING TYPES ------------------')
    # type_checker = TypeChecker(context, errors)
    # type_checker.visit(ast)
    # if debug:
    #     print('Errors: [')
    #     for error in errors:
    #         print('\t', error)
    #     print(']')
        # print('Context:')
        # print(context)
        # print('Scope:')
        # print(scope)
    return ast, errors, context, scope


In [ ]:
print(semantic_analysis_pipeline(asts[5],debug=True))

In [ ]:
print(semantic_analysis_pipeline(asts[7],debug=True))